In [5]:
import pandas as pd
import re
import string
import pickle

In [6]:
data_dir = '/home/mahbub/Desktop/sepsis_text_IoMT_project/data/embedding_generation_data/'

In [232]:
def calculate_start_end_time(file_name):
    
    df = pd.read_csv(file_name)
    
    column_list = ['start_time', 'end_time', 'icustay_id', 'subject_id']
    
    df= df[df.columns & column_list]
    
    #df['start_time'] = pd.to_datetime(df['start_time'])
    #df['end_time'] = pd.to_datetime(df['end_time'])
    
    df_end_time = df.groupby(['icustay_id', 'subject_id'])[["end_time"]].max()
    
    df_start_time = df.groupby(['icustay_id', 'subject_id'])[["start_time"]].min()
    
    df = df_end_time.merge(df_start_time, on=['icustay_id', 'subject_id'], how = 'inner')
    
    print(len(df))
    #print(len(patient_list))
    
    return df

In [113]:
def preprocess1(x):
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('admission date:','',y)
    y=re.sub('discharge date:','',y)
    y=re.sub('--|__|==','',y)
    
    # remove, digits, spaces
    y = y.translate(str.maketrans("", "", string.digits))
    y = " ".join(y.split())
    return y

def preprocessing(df_notes): 
    df_notes['TEXT']=df_notes['TEXT'].fillna(' ')
    df_notes['TEXT']=df_notes['TEXT'].str.replace('\n',' ')
    df_notes['TEXT']=df_notes['TEXT'].str.replace('\r',' ')
    df_notes['TEXT']=df_notes['TEXT'].apply(str.strip)
    df_notes['TEXT']=df_notes['TEXT'].str.lower()

    df_notes['TEXT']=df_notes['TEXT'].apply(lambda x: preprocess1(x))
    
    return df_notes


In [233]:
test_df = calculate_start_end_time(data_dir+'test_data_1_hours.csv')
tune_df = calculate_start_end_time(data_dir+'tune_data_1_hours.csv')
train_df = calculate_start_end_time(data_dir+'train_data_1_hours.csv')

/var/folders/2z/6h8_bk9j3vncns4t9qdn41rw0000gq/T/ipykernel_1288/768403759.py:9: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  df= df[df.columns & column_list]


1824
1796
14334


In [234]:
patient_df = pd.concat([test_df, tune_df, train_df])

print( len(patient_df) - (len(test_df)+len(tune_df)+len(train_df)))

0


In [236]:
patient_df.to_csv(data_dir+'sepsis_start_end_time.csv')

In [237]:
patient_df =pd.read_csv(data_dir+'sepsis_start_end_time.csv')

In [239]:
patient_df['start_time']= pd.to_datetime(patient_df['start_time'])
patient_df['end_time']= pd.to_datetime(patient_df['end_time'])

In [241]:
patient_list = patient_df['subject_id'].unique().tolist()
print(len(patient_list))

14404


In [242]:
start_end_time_dict = dict()

for subject_id in patient_list:
                       
    start_time_list = patient_df.loc[patient_df['subject_id'] == subject_id]['start_time'].tolist()
    end_time_list = patient_df.loc[patient_df['subject_id'] == subject_id]['end_time'].tolist()
    icustay_id_list = patient_df.loc[patient_df['subject_id'] == subject_id]['icustay_id'].tolist()
    
    start_end_time_dict[subject_id] = [start_time_list, end_time_list, icustay_id_list]
    

In [64]:
#(start_end_time_dict[27597])

In [244]:
with open(data_dir+'start_end_time_dict.pkl', 'wb') as f:
    pickle.dump(start_end_time_dict, f)

In [7]:
with open(data_dir+'start_end_time_dict.pkl', 'rb') as f:
    start_end_time_dict = pickle.load(f)

In [ ]:
all_notes_df = pd.read_csv(data_dir+'NOTEEVENTS.csv')

In [ ]:
sepsis_df = all_notes_df.loc[all_notes_df['SUBJECT_ID'].isin(patient_list)]

In [ ]:
clean_sepsis_df = preprocessing(sepsis_df)

print(len(clean_sepsis_df))

In [ ]:
clean_sepsis_df.to_csv(data_dir+'sepsis_text.csv')

In [121]:
clean_sepsis_df = pd.read_csv(data_dir+'sepsis_text.csv')

/Users/dsv/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [122]:
clean_sepsis_df.drop(clean_sepsis_df.index[clean_sepsis_df['CATEGORY'] == 'Discharge summary'], inplace = True)

In [123]:
clean_sepsis_df['CATEGORY'].unique().tolist()#CATEGORY

['Echo',
 'ECG',
 'Nursing',
 'Physician ',
 'Rehab Services',
 'Respiratory ',
 'Nutrition',
 'Social Work',
 'General',
 'Case Management ',
 'Pharmacy',
 'Consult',
 'Radiology',
 'Nursing/other']

In [124]:
column_list = ['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR']
    
clean_sepsis_df= clean_sepsis_df[clean_sepsis_df.columns & column_list]

/var/folders/2z/6h8_bk9j3vncns4t9qdn41rw0000gq/T/ipykernel_1288/4273257310.py:3: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  clean_sepsis_df= clean_sepsis_df[clean_sepsis_df.columns & column_list]


In [159]:
clean_sepsis_df['CHARTDATE'] = pd.to_datetime(clean_sepsis_df['CHARTDATE'])
clean_sepsis_df['CHARTTIME'] = pd.to_datetime(clean_sepsis_df['CHARTTIME'])
clean_sepsis_df['STORETIME'] = pd.to_datetime(clean_sepsis_df['STORETIME'])

In [175]:
clean_sepsis_df.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME',
       'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR'],
      dtype='object')

In [295]:
text_start_end_time_dict = dict()

for subject_id in patient_list:
                       
    CHARTDATE_list = clean_sepsis_df.loc[clean_sepsis_df['SUBJECT_ID'] == subject_id]['CHARTDATE'].tolist()
    CHARTTIME_list = clean_sepsis_df.loc[clean_sepsis_df['SUBJECT_ID'] == subject_id]['CHARTTIME'].tolist()
    STORETIME_list = clean_sepsis_df.loc[clean_sepsis_df['SUBJECT_ID'] == subject_id]['STORETIME'].tolist()
    HADM_ID_list = clean_sepsis_df.loc[clean_sepsis_df['SUBJECT_ID'] == subject_id]['HADM_ID'].tolist()
    ROW_ID_list = clean_sepsis_df.loc[clean_sepsis_df['SUBJECT_ID'] == subject_id]['ROW_ID'].tolist()
    
    text_start_end_time_dict[subject_id] = [CHARTDATE_list, CHARTTIME_list, STORETIME_list, HADM_ID_list, ROW_ID_list]

In [297]:
with open(data_dir+'text_start_end_time_dict.pkl', 'wb') as f:
    pickle.dump(text_start_end_time_dict, f)

In [8]:
with open(data_dir+'text_start_end_time_dict.pkl', 'rb') as f:
    text_start_end_time_dict = pickle.load(f)

In [36]:
time_window=1

In [37]:
text_time_map_icu_id_dict = dict()

In [38]:
time_list=[]
subject_list=[]
HADM_list=[]
charttime_bool_list=[]
row_list=[]

for subject_id in start_end_time_dict: #[27597]:#start_end_time_dict:
    
    if subject_id in text_start_end_time_dict:
        
        start_time_list = start_end_time_dict[subject_id][0]
        end_time_list = start_end_time_dict[subject_id][1]
        icustay_id_list = start_end_time_dict[subject_id][2]

        CHARTDATE_list = text_start_end_time_dict[subject_id][0]
        CHARTTIME_list = text_start_end_time_dict[subject_id][1]
        STORETIME_list = text_start_end_time_dict[subject_id][2]
        HADM_ID_list = text_start_end_time_dict[subject_id][3]
        ROW_ID_list =text_start_end_time_dict[subject_id][4]
        
        #print(HADM_ID_list)
        
        for icu_index in range(len(icustay_id_list)):
        
            start_time = start_time_list[icu_index]
            end_time = end_time_list[icu_index]
            icustay_id = icustay_id_list[icu_index]
            
            time_list=[]
            subject_list=[]
            HADM_list=[]
            charttime_bool_list=[]
            row_list=[]
            
            for text_index in range(len(HADM_ID_list)):
    
                CHARTDATE = CHARTDATE_list[text_index]
                CHARTTIME = CHARTTIME_list[text_index]
                STORETIME = STORETIME_list[text_index]
                HADM_ID = HADM_ID_list[text_index]
                row_id=ROW_ID_list[text_index]
                
                if (pd.isna(CHARTTIME) == False) and (CHARTTIME >= start_time) and (CHARTTIME <= (end_time-pd.DateOffset(hours=time_window))):
                    
                    time_list.append(CHARTTIME)
                    subject_list.append(subject_id)
                    HADM_list.append(HADM_ID)
                    row_list.append(row_id)
                    charttime_bool_list.append(True)
                    
                elif pd.isna(CHARTTIME) and (pd.isna(CHARTDATE) == False) and (CHARTDATE.date() >= start_time) and (CHARTDATE.date() <= end_time):
                    
                    time_list.append(CHARTDATE)
                    subject_list.append(subject_id)
                    HADM_list.append(HADM_ID)
                    row_list.append(row_id)
                    charttime_bool_list.append(False)
                    
            text_time_map_icu_id_dict[icustay_id]=[time_list, subject_list, HADM_list, charttime_bool_list, row_list]
                    
                
                
        

/var/folders/2z/6h8_bk9j3vncns4t9qdn41rw0000gq/T/ipykernel_820/2027307178.py:51: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  elif pd.isna(CHARTTIME) and (pd.isna(CHARTDATE) == False) and (CHARTDATE.date() >= start_time) and (CHARTDATE.date() <= end_time):


In [39]:
with open(data_dir+'text_time_map_icu_id_dict.pkl', 'wb') as f:
    pickle.dump(text_time_map_icu_id_dict, f)

In [9]:
with open(data_dir+'text_time_map_icu_id_dict.pkl', 'rb') as f:
    text_time_map_icu_id_dict = pickle.load(f)

In [ ]:
#text_time_map_icu_id_dict[245733]

In [11]:
sepsis_non_text_df = pd.concat([pd.read_csv(data_dir+'test_data_1_hours.csv'), pd.read_csv(data_dir+'tune_data_1_hours.csv'), pd.read_csv(data_dir+'train_data_1_hours.csv')]) 
sepsis_non_text_df['start_time']= pd.to_datetime(sepsis_non_text_df['start_time'])
sepsis_non_text_df['end_time']= pd.to_datetime(sepsis_non_text_df['end_time'])
icustay_id_list = sepsis_non_text_df['icustay_id'].unique().tolist()

In [17]:

non_text_row_ids=sepsis_non_text_df.loc[sepsis_non_text_df['icustay_id'] == 231472]['row_id'].tolist()
print(non_text_row_ids)

[88105, 88106, 88107, 88108, 88109, 88110]


In [59]:
294125 in text_time_map_icu_id_dict

True

In [12]:
non_text_row_id_mapping_text_row_id_dict = dict()

In [13]:
text_row_id_list = []
text_bool_list=[]

for icustay_id in icustay_id_list:#[231472]:#icustay_id_list:
    
    if icustay_id in text_time_map_icu_id_dict: #[245733]:#
        
        text_timestamps=text_time_map_icu_id_dict[icustay_id][0]
        text_row_ids=text_time_map_icu_id_dict[icustay_id][4]
        text_chart_date_bools=text_time_map_icu_id_dict[icustay_id][3]
        
        non_text_start_times=sepsis_non_text_df.loc[sepsis_non_text_df['icustay_id'] == icustay_id]['start_time'].tolist()
        non_text_end_times=sepsis_non_text_df.loc[sepsis_non_text_df['icustay_id'] == icustay_id]['end_time'].tolist()
        non_text_row_ids=sepsis_non_text_df.loc[sepsis_non_text_df['icustay_id'] == icustay_id]['row_id'].tolist()

        for non_text_index in range(len(non_text_start_times)):
    
            non_text_start_time = non_text_start_times[non_text_index]
            non_text_end_time = non_text_end_times[non_text_index]
            non_text_row_id = non_text_row_ids[non_text_index]
    
            text_row_id_list = []
            text_bool_list=[]
    
            for text_index in range(len(text_timestamps)):
        
                text_timestamp = text_timestamps[text_index]
                text_row_id = text_row_ids[text_index]
                text_chart_date_bool = text_chart_date_bools[text_index]
        
                if text_chart_date_bool:
                    text_timestamp = text_timestamp.date()
            
        
                if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
            
                    text_row_id_list.append(text_row_id)
                    text_bool_list.append(text_chart_date_bool)
                    
            non_text_row_id_mapping_text_row_id_dict[non_text_row_id] = [text_row_id_list, text_bool_list]


/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() =

/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() =

/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() =

/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() =

/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() =

...

/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() =

/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() =

/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() =

/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() =

/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if text_timestamp >= non_text_start_time and text_timestamp <= non_text_end_time:
/tmp/ipykernel_5189/2276137115.py:35: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() =

In [18]:
non_text_row_id_mapping_text_row_id_dict[88105]

[[1169768, 1169763], [True, True]]

In [20]:
with open(data_dir+'non_text_row_id_mapping_text_row_id_dict.pkl', 'wb') as f:
    pickle.dump(non_text_row_id_mapping_text_row_id_dict, f)

In [34]:
with open('/home/mahbub/Desktop/sepsis_text_IoMT_project/data/embedding_generation_data/non_text_row_id_mapping_text_row_id_dict.pkl', 'rb') as f:
    f = pickle.load(f)

In [61]:
f[37485]

[[681381, 681382, 681386, 681378], [True, True, True, True]]

In [62]:
768*4

3072